**Lorenzo Caimi**  
Matricola: 800052

# Project

All groups and individual must do the following:

   
   1. <a href="#1"> Convert the app sizes to a number </a>
   2. <a href="#2"> Convert the number of installs to a number </a>
   3. <a href="#3"> Transform “Varies with device” into a missing value </a>
   4. <a href="#4"> Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2) </a>
   5. <a href="#5"> Remove the duplicates </a>
   6. <a href="#6"> For each category, compute the number of apps </a>
   7. <a href="#7"> For each category, compute the average rating </a>
   8. <a href="#8"> Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity </a>
   9. <a href="#9"> For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre) </a>
   10. <a href="#10"> For each genre, compute the average rating. What is the genre with highest average? </a>
   11. <a href="#11"> For each app, compute the approximate income, obtain as a product of number of installs and price. </a>
   12. <a href="#12"> For each app, compute its minimum and maximum Sentiment_polarity </a>


## Carico i Dataset

In [1]:
import pandas as pd
import numpy as np
import re

gps = pd.read_csv('C:\\Users\\Utente\\Desktop\\Data Science\\Foundations of Computer Science\\googleplaystore.csv')
gpsur = pd.read_csv('C:\\Users\\Utente\\Desktop\\Data Science\\Foundations of Computer Science\\googleplaystore_user_reviews.csv')

## Analisi preliminari

In [2]:
gps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [3]:
gpsur.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [4]:
gps = gps.drop(gps.index[10472])

##  1. <a name="1">  Convert the app sizes to a number </a>

Si nota la presenza di:  
`M` che indica 1 milione  
`k` che indica 1 migliaio  
`Varies with device` che non sappiamo come interpretarlo.  

Si procederà quindi alla rimozione delle righe in cui è presente l'attributo `Varies with device` .  

In [27]:
(gps['Size'] == 'Varies with device').value_counts()
gps = gps[gps['Size'] != 'Varies with device'] 
gps = gps.reset_index(drop=True)

Trasformo ora in `int`

In [6]:
decimali = pd.to_numeric(gps['Size'].replace(['[0-9]','[.]','M','k'],['','','1000000','1000'],regex=True) )                                
numeri = pd.to_numeric(gps['Size'].replace(['M','k'],['',''],regex=True))

gps['Size'] = numeri*decimali
gps['Size'] = gps['Size'].round(0).astype(int)

gps['Size'].head()

0    19000000
1    14000000
2     8700000
3    25000000
4     2800000
Name: Size, dtype: int32

## 2. <a name="2"> Convert the number of installs to a number </a>

In [7]:
gps['Installs'] = (gps['Installs'].replace([',','[+]'],['',''], regex=True))
gps['Installs'] = pd.to_numeric(gps['Installs'])

gps['Installs'].head()

0       10000
1      500000
2     5000000
3    50000000
4      100000
Name: Installs, dtype: int64

## 3. <a name="3"> Transform “Varies with device” into a missing value </a>


In [25]:
gps[gps == 'Varies with device'] = np.nan

gps[gps == "Varies with device"].count()

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       0
Android Ver       0
Income            0
dtype: int64

## 4. <a name="4"> Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2) </a>

In [9]:
gps['Current Ver'] = gps['Current Ver'].replace(["^[\.]","[^\d\.]","[\.]$"], ["", "", ""], regex=True)
gps['Android Ver'] = gps['Android Ver'].replace(["^[\.]","[^\d\.]","[\.]$"], ["", "", ""], regex=True)

gps = gps.replace(r'^\s*$', np.nan, regex=True) 

## 5. <a name= "5"> Remove the duplicates </a>

In [10]:
gps.drop_duplicates(inplace = True)
gps = gps.reset_index(drop=True)

gpsur.drop_duplicates(inplace = True)
gpsur = gpsur.reset_index(drop=True)

## 6. <a name="6"> For each category, compute the number of apps </a>

In [11]:
Cat_App = gps.groupby('Category',as_index=False)['App'].count()
Cat_App = Cat_App.rename(columns={'App':'App_Count'})

Cat_App.head()

,Category,App_Count
0,ART_AND_DESIGN,62
1,AUTO_AND_VEHICLES,75
2,BEAUTY,47
3,BOOKS_AND_REFERENCE,196
4,BUSINESS,379


## 7. <a name="7">For each category, compute the average rating </a>

In [12]:
Cat_Rat = gps.groupby('Category',as_index=False)['Rating'].mean()
Cat_Rat = Cat_Rat.rename(columns={'Rating':'Rating_Avg'})

Cat_Rat.head()

,Category,Rating_Avg
0,ART_AND_DESIGN,4.361017
1,AUTO_AND_VEHICLES,4.147619
2,BEAUTY,4.291892
3,BOOKS_AND_REFERENCE,4.321678
4,BUSINESS,4.100000


## 8. <a name="8"> Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity </a>

In [13]:
duplicati = gps['Genres'].str.split(";", expand = True) 

gps_A = gps.copy()
gps_A['Genres'] = duplicati[[0]]
gps_A = gps_A[['App','Genres']]

gps_B = gps.copy()
gps_B['Genres'] = duplicati[[1]]
gps_B = (gps_B[['App','Genres']].dropna()).reset_index(drop=True)

Finale = pd.concat([gps_A,gps_B])
Finale.drop_duplicates(inplace=True) 
Finale = Finale.reset_index(drop=True)

Finale[Finale['App'] == 'Pixel Draw - Number Art Coloring Book']

,App,Genres
4,Pixel Draw - Number Art Coloring Book,Art & Design
8439,Pixel Draw - Number Art Coloring Book,Creativity


## 9. <a name="9"> For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre) </a>

In [14]:
lista = gps['Genres'].unique()

Generi = pd.DataFrame(index = range(0,len(gps)), columns=lista)

for i in range(0,len(lista)):
    Generi[Generi.columns[i]]='False'

for i in range(0,len(lista)):
    for x in range(0,len(lista)):
        if gps['Genres'][i] == Generi.columns[x]:
            Generi[Generi.columns[x]][i] = 'True'

gps_Completo = pd.concat([gps,Generi],axis=1)


In [15]:
Generi.head()

,Art & Design,Art & Design;Pretend Play,Art & Design;Creativity,Art & Design;Action & Adventure,Auto & Vehicles,Beauty,Books & Reference,Business,Comics,Comics;Creativity,...,Entertainment;Creativity,Sports;Action & Adventure,Books & Reference;Creativity,Books & Reference;Education,Puzzle;Education,Role Playing;Education,Role Playing;Brain Games,Strategy;Education,Racing;Pretend Play,Strategy;Creativity
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [16]:
gps_Completo.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Entertainment;Creativity,Sports;Action & Adventure,Books & Reference;Creativity,Books & Reference;Education,Puzzle;Education,Role Playing;Education,Role Playing;Brain Games,Strategy;Education,Racing;Pretend Play,Strategy;Creativity
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,10000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,False
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,500000,Free,0,Everyone,Art & Design;Pretend Play,...,False,False,False,False,False,False,False,False,False,False
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,5000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,False
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,50000000,Free,0,Teen,Art & Design,...,False,False,False,False,False,False,False,False,False,False
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,100000,Free,0,Everyone,Art & Design;Creativity,...,False,False,False,False,False,False,False,False,False,False


## 10. <a name="10"> For each genre, compute the average rating. What is the genre with highest average? <a>

In [17]:
nome = (gps.groupby('Genres').Rating.mean()).idxmax()
rating = (gps.groupby('Genres').Rating.mean()).max()
print('Genre: ',nome,'\nAvg Rating: ',rating)

Genre:  Board;Pretend Play 
Avg Rating:  4.8


## 11. <a name="11"> For each app, compute the approximate income, obtain as a product of number of installs and price. </a>

`Installs` è stato trasformato in intero nel punto 2.  
Devo fare la stessa cosa con `Price`

In [19]:
gps['Price'] = (gps['Price'].replace('[$]',' ', regex=True))
gps['Price'] = pd.to_numeric(gps['Price'])

gps = gps.copy()
gps['Income'] = gps['Price']*gps['Installs']

gps[gps['Income'] != 0].head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Income
186,TurboScan: scan documents and receipts in PDF,BUSINESS,4.7,11442,6800000,100000,Paid,4.99,Everyone,Business,"March 25, 2018",1.5.2,4.0,499000.0
187,Tiny Scanner Pro: PDF Doc Scan,BUSINESS,4.8,10295,39000000,100000,Paid,4.99,Everyone,Business,"April 11, 2017",3.4.6,3.0,499000.0
310,Calculator,DATING,2.6,57,6200000,1000,Paid,6.99,Everyone,Dating,"October 25, 2017",1.1.6,4.0,6990.0
311,Truth or Dare Pro,DATING,NaN,0,20000000,50,Paid,1.49,Teen,Dating,"September 1, 2017",1.0,4.0,74.5
312,"Private Dating, Hide App- Blue for PrivacyHider",DATING,NaN,0,18000,100,Paid,2.99,Everyone,Dating,"July 25, 2017",1.0.1,4.0,299.0


## 12. <a name="12"> For each app, compute its minimum and maximum Sentiment_polarity </a>

In [20]:
gpsur = gpsur.dropna()
gpsur = gpsur.reset_index(drop=True)

In [28]:
massimo = gpsur.groupby('App',as_index=False)['Sentiment_Polarity'].max()
massimo = massimo.rename(columns={'Sentiment_Polarity':'Sentiment_Polarity_Max'})

minimo = gpsur.groupby('App',as_index=False)['Sentiment_Polarity'].min()
minimo = minimo.rename(columns={'Sentiment_Polarity':'Sentiment_Polarity_Min'})

unione = pd.merge(massimo,minimo, on='App')
unione = unione.dropna()

unione.head()

,App,Sentiment_Polarity_Max,Sentiment_Polarity_Min
0,10 Best Foods for You,1.000000,-0.8000
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.910000,-0.1125
2,11st,1.000000,-1.0000
3,1800 Contacts - Lens Store,0.838542,-0.3000
4,1LINE – One Line with One Touch,1.000000,-0.8250
